In [1]:
import os
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

from itertools import cycle
from collections import Counter
from shutil import copyfile
from PIL import Image
from IPython import display 

%matplotlib inline

In [2]:
def compute_IoU(pred, mask, trigger=0.999):
    pred = (pred>trigger)
    mask = (mask>0.9)
    union = np.logical_or(pred, mask)
    intersection = np.logical_and(pred, mask)
    return intersection.sum()/union.sum()

In [5]:
# for idx, img_path in enumerate(glob.glob("images/prepared/train/images/*.png")):
#     img_name = os.path.basename(img_path)
#     mask_path = os.path.join("images/prepared/train/masks", img_name)
    
#     image = cv2.imread(img_path)[:, :, 1]
#     print(image.shape)
#     th3 = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
#     cv2.imwrite("images/prepared/train/images3/" + img_name, th3)

(256, 256)
(256, 256)
(256, 320)
(256, 320)
(256, 320)
(256, 256)
(256, 256)
(360, 360)
(256, 256)
(360, 360)
(360, 360)
(260, 347)
(256, 256)
(256, 256)
(360, 360)
(256, 256)
(256, 256)
(360, 360)
(256, 256)
(256, 256)
(256, 256)
(520, 696)
(360, 360)
(1024, 1024)
(256, 256)
(1024, 1024)
(256, 256)
(256, 256)
(256, 256)
(520, 696)
(256, 256)
(256, 256)
(256, 320)
(256, 256)
(256, 320)
(256, 256)
(360, 360)
(256, 320)
(256, 256)
(256, 320)
(256, 320)
(256, 256)
(520, 696)
(360, 360)
(256, 256)
(520, 696)
(256, 256)
(520, 696)
(256, 256)
(256, 320)
(256, 256)
(256, 256)
(256, 320)
(512, 640)
(360, 360)
(256, 320)
(520, 696)
(360, 360)
(256, 320)
(256, 256)
(256, 256)
(520, 696)
(256, 256)
(520, 696)
(260, 347)
(1024, 1024)
(360, 360)
(256, 256)
(256, 256)
(1024, 1024)
(520, 696)
(256, 256)
(360, 360)
(256, 256)
(256, 256)
(520, 696)
(256, 320)
(256, 256)
(256, 256)
(256, 256)
(256, 320)
(256, 256)
(256, 256)
(256, 320)
(256, 256)
(360, 360)
(520, 696)
(256, 256)
(256, 256)
(603, 1272)
(

In [3]:
h, w, c = 512, 512, 3
BATCH_SIZE = 3
NB_IMAGES_TRAIN = 603
NB_IMAGES_TEST = 67
EPOCHS = 50
SEED = 42

In [4]:
img_id_train = []
img_id_test = []
for idx, img_path in enumerate(glob.glob("images/prepared/train/images/*.png")):
    if idx % 10 == 0:
        img_id_test.append(os.path.basename(img_path))
    else:
        img_id_train.append(os.path.basename(img_path))

In [5]:
def generator_images(list_image, batch_size = 3):
    gen = cycle(list_image)
    while True:
        X1 = np.zeros((batch_size, 512, 512, 1))
        X2 = np.zeros((batch_size, 512, 512, 3))
        X3 = np.zeros((batch_size, 512, 512, 1))
        Y = np.zeros((batch_size, 512, 512, 1))
        for i in range(batch_size):
            img_name = next(gen)
            image = cv2.imread("images/prepared/train/images/"+img_name)[:, :, :1].astype(np.float32)
            image2 = cv2.imread("images/prepared/train/images2/"+img_name).astype(np.float32)
            image3 = cv2.imread("images/prepared/train/images3/"+img_name)[:, :, :1].astype(np.float32)
            mask = cv2.imread("images/prepared/train/masks/"+img_name)[:, :, :1].astype(np.float32)
            
#             print(image.shape)
#             print(image2.shape)
#             print(image3.shape)
#             print(mask.shape)
            
            image /= 255.
            image2 /= 255.
            image3 /= 255.
            mask /= 255.
            
#             print((image).min(), (image).max())
#             print((image2).min(), (image2).max())
#             print((image3).min(), (image3).max())
#             print((mask).min(), (mask).max())
            
            image = cv2.resize(image, (512, 512))
            image2 = cv2.resize(image2, (512, 512))
            image3 = cv2.resize(image3, (512, 512))
            mask = cv2.resize(mask, (512, 512))
            
#             print(image.shape)
#             print(image2.shape)
#             print(image3.shape)
#             print(mask.shape)
            
            X1[i] = np.expand_dims(image, 2)
            X2[i] = image2
            X3[i] = np.expand_dims(image3, 2)
            Y[i] = np.expand_dims(mask, 2)

        yield [X1, X3, X2], Y
        
train_gen = generator_images(img_id_train)
test_gen = generator_images(img_id_test)

In [6]:
from models import model_extended as md

model = md.create_model(h, w, c)

Using TensorFlow backend.


In [7]:
model.fit_generator(generator = train_gen,
                    steps_per_epoch = NB_IMAGES_TRAIN//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=test_gen,
                    validation_steps = NB_IMAGES_TEST//BATCH_SIZE,
#                     callbacks = [predictions]
                    )

Epoch 1/50
201/201 [==============================] - 48s 240ms/step - loss: 0.1525 - binary_crossentropy: 0.1525 - val_loss: 0.0989 - val_binary_crossentropy: 0.0989
Epoch 2/50
 30/201 [===>..........................] - ETA: 35s - loss: 0.1024 - binary_crossentropy: 0.1024

KeyboardInterrupt: 

In [6]:
model.save("models/model_extended_large.h5")
model.save_weights('models/model_extended_weights_large.h5')

In [13]:
result_trigger = []

for trigger in [0.05 * i for i in range(5, 15)]:
    final_score = []
    threshold = [0.05 * i for i in range(10, 20)]
    for t in threshold:
        score = []
        test_gen = generator_images(img_id_test, 1)
        for X, Y in test_gen:
            y_pred = model.predict(X)

            mask = Y[0][0, :, :, 0]
            pred = y_pred[0, :, :, 0]

            IOU = compute_IoU(pred, mask, trigger=trigger)  # if predicition of a specific pixel is above trigger, it is white
            if IOU >= t:
                score.append(1)
            else:
                score.append(0)
        final_score.append(sum(score)/len(score))
#         print("Score (with threshold = {:.2f}) = {:.3f}".format(t, sum(score)/len(score)))
    print("Final score = {:.3f}".format(sum(final_score)/len(final_score)))
    result_trigger.append(sum(final_score)/len(final_score))

KeyboardInterrupt: 